In [1]:
pip install faiss-cpu 


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy requests


   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   -------- ------------------------------- 1/5 [idna]
   -------- ------------------------------- 1/5 [idna]
   ---------------- ----------------------- 2/5 [charset_normalizer]
   ------------------------ --------------- 3/5 [certifi]
   -------------------------------- ------- 4/5 [requests]
   ---------------------------------------- 5/5 [requests]

Note: you may need to restart the kernel to use updated packages.


In [3]:
data = """
Making an Impact
Helping Millions of Students Succeed
Sudhanshu's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, iNeuron has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students, like Sudhanshu himself, came from disadvantaged backgrounds. They saw iNeuron as a lifeline—an opportunity to rise above their circumstances.

In 2022, iNeuron was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, Sudhanshu remained focused on his mission. Even after the acquisition, iNeuron continued to offer some of the most affordable and accessible tech courses in the world.

The Entrepreneur and Teacher: Sudhanshu's Dual Legacy
Sudhanshu's journey isn't just one of entrepreneurial success; it's also a story of dedication to teaching. Throughout his career, he has remained a passionate educator, constantly looking for ways to empower others through knowledge. Whether teaching courses in Big Data, Data Science, or programming, Sudhanshu has always sought to make complex subjects accessible to learners at all levels.

His commitment to affordable education has earned him the respect and admiration of countless students. Many credit Sudhanshu with changing their lives, helping them secure jobs, improve their skills, and break free from the limitations of their backgrounds."""

In [4]:
clean_data=data.strip()

In [9]:
max_char = 300
overlap = 100
chunks=[]
i=0
while i<len(clean_data):
    piece = clean_data[i:i+max_char]
    chunks.append(piece)
    i=i+max_char-overlap


In [11]:
len(chunks)
chunks

["Making an Impact\nHelping Millions of Students Succeed\nSudhanshu's commitment to affordable education wasn't just a business strategy—it was his life's mission. Over the years, iNeuron has helped over 1.5 million students from 34+ countries, providing them with the skills they need to succeed in toda",
 "1.5 million students from 34+ countries, providing them with the skills they need to succeed in today's competitive job market. Many of these students, like Sudhanshu himself, came from disadvantaged backgrounds. They saw iNeuron as a lifeline—an opportunity to rise above their circumstances.\n\nIn 20",
 'backgrounds. They saw iNeuron as a lifeline—an opportunity to rise above their circumstances.\n\nIn 2022, iNeuron was acquired by PhysicsWallah in a deal worth ₹250 crore. While this acquisition was a significant milestone, Sudhanshu remained focused on his mission. Even after the acquisition, iNeuron',
 "ignificant milestone, Sudhanshu remained focused on his mission. Even after t

In [12]:
import requests
import numpy as np

def generate_embeddings(text):
    url = "https://api.euron.one/api/v1/euri/embeddings"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer *******"
    }
    payload = {
        "input": text,
        "model": "text-embedding-3-small"
    }

    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    
    embedding = np.array(data['data'][0]['embedding'])
    
    return embedding


In [14]:
for i in chunks:
    embedding = generate_embeddings(i)
    print(embedding)

[ 0.01636502 -0.01330146  0.04360536 ... -0.00716132 -0.00025957
  0.00777273]
[-0.02781193  0.00343053  0.04798149 ... -0.01521909 -0.01743826
 -0.00741914]
[-0.02119237 -0.03513358  0.01877532 ... -0.01876093  0.00953153
  0.02064566]
[ 0.00011691 -0.01620286  0.03854437 ... -0.03406553  0.00343817
  0.02559526]
[ 0.00538704 -0.01142304  0.04874471 ... -0.0355971   0.00306157
  0.00572053]
[-0.00912461 -0.01832682  0.04314627 ... -0.02185768 -0.00779892
 -0.01152378]
[-0.00252494 -0.02891203  0.04281055 ... -0.00076008 -0.01947349
  0.00868658]
[ 0.0317256   0.01637366  0.04584625 ... -0.01135022 -0.06958151
  0.01558773]


In [16]:
emb_list=[]
meta=[]
for idx,chunk in enumerate(chunks):
    vec = generate_embeddings(chunk)
    emb_list.append(vec.astype("float32"))

    meta.append({"id":idx,"test":chunk})
    

In [18]:
xb=np.vstack(emb_list)

In [19]:
import faiss

In [20]:
faiss.normalize_L2(xb)

In [24]:
d=xb.shape[1]
d

1536

In [27]:
index=faiss.IndexFlatIP(d)
index.add(xb)
index

<faiss.swigfaiss_avx2.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x000001DF76F1D7D0> >

In [28]:
index_path="index_mk.faiss"
meta_path="meta_mk.jsonl"

In [29]:
faiss.write_index(index,index_path)

In [30]:
import json,os

In [32]:
with open(meta_path,"w") as f:
    for item in meta:
        f.write(json.dumps(item)+"\n")

In [35]:
query = "who is Sudhanshu?"
q=generate_embeddings(query).astype("float32").reshape(1,-1)
faiss.normalize_L2(q)
index.search(q,5)

(array([[0.55319285, 0.5195153 , 0.5064372 , 0.49625516, 0.40612662]],
       dtype=float32),
 array([[4, 6, 5, 3, 1]]))